In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget

from matplotlib import collections as mc
import cmath
from matplotlib.patches import Ellipse
from math import sqrt

from numpy.linalg import eig


## Linear Interpolation Using Vectors

In [ ]:
fix, ax = plt.subplots(figsize=(6,4))
ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.grid(True)

def linear_interpolate(x,y, theta):
    return x + theta * (y - x)

@widgets.interact(theta=(0.0, 1.0, 0.01))
def update(theta = 0.0):
    [l.remove() for l in ax.collections]
    z1 = np.asarray([0.0, 0.0])
    z2 = np.asarray([1.0, 1.0])
    z = linear_interpolate(z1, z2, theta)
    ax.scatter(z[0], z[1], color="C0")

## Interpolating Angles Using Complex Numbers

In [ ]:
fix, ax = plt.subplots(figsize=(6,6))
ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.grid(True)

def complex_interpolate(x, y, alpha):
    x_complex = x[0] + 1j * x[1]
    y_complex = y[0] + 1j * y[1]    
    z = cmath.exp(1j* alpha * (cmath.phase(y_complex) - cmath.phase(x_complex))) * (x_complex)
    return np.asarray([z.real, z.imag])

@widgets.interact(alpha=(0.0, 1.0, 0.01))
def update(alpha = 0.0):
    [l.remove() for l in ax.collections]
    z1 = np.asarray([1.0, 0.0])
    z2 = np.asarray([0.0, 1.0])

    z = complex_interpolate(z1, z2, alpha)
    ax.scatter(z[0], z[1], color="C0")

## Lie Group Definitions

Probably the best single reference for these and related formulas is Ethan Eade's notes:

http://ethaneade.com/lie.pdf

In [ ]:
class so2(object):
    def __init__(self):
        pass
    
    @staticmethod
    def exp(theta):
        '''
        Map x to a lie group element. We assume theta is a scalar and by skip the step where 
        we construct the skew-symmetric matrix [[0, -theta],[theta, 0]].
        '''
        return np.asarray([[np.cos(theta), -np.sin(theta)],
                           [np.sin(theta), np.cos(theta)]])
    
class SO2(object):
    def __init__(self):
        pass
    
    @staticmethod
    def log(X):
        '''
        Find a lie algebra element that generates x.
        '''
        return np.arctan2(X[1,0], X[0,0])
    
    @staticmethod
    def inv(X):
        return np.transpose(X)
    
    lie_algebra = so2
    


In [ ]:
def skew_2(omega):
    return np.asarray([[0.0, -omega],
                       [omega, 0.0]])

def skew_3(omega):
    return np.asarray([[0, -omega[2],omega[1]],
                       [omega[2],0,-omega[0]],
                       [-omega[1],omega[0],0]])

In [ ]:
class se2(object):
    def __init__(self):
        pass
    
    @staticmethod
    def exp(delta):
        theta = delta[0]
        u = delta[1:3]
        #R = np.asarray([[np.cos(theta), -np.sin(theta)],
        #                [np.sin(theta), np.cos(theta)]])
        R = so2.exp(theta)
        
        # If theta is small, use a truncated Taylor series to approximate V.
        # Otherwise use the closed-form expression.
        if theta < 0.001:
            V = np.identity(2) + (1.0 - (theta**2)/6.0 + (theta**4)/120.0) * skew_2(theta) + (0.5 - (theta**2)/24.0 + (theta**4)/720.0) * skew_2(theta) @ skew_2(theta)
        else:
            V = (1.0/theta) * np.asarray([[np.sin(theta), -(1.0 - np.cos(theta))],
                                          [1.0 - np.cos(theta), np.sin(theta)]])
        
        t = V @ u
        return np.asarray([[R[0,0], R[0,1], t[0]],
                           [R[1,0], R[1,1], t[1]],
                           [0.0,    0.0,    1.0]])

class SE2(object):
    lie_algebra = se2
    
    def __init__(self):
        pass
    
    @staticmethod
    def log(X):
        theta = np.arctan2(X[1,0], X[0,0])

        if theta < 0.001:
            V = np.identity(2)
            V_inv = V
        else:
            A = np.sin(theta)/theta
            B = (1.0 - np.sin(theta))/theta
            V_inv = (1.0 / (A**2 + B**2)) * np.asarray([[A, B], 
                                                        [-B, A]])
        t = X[0:2, 2]
        u = V_inv @ t
        return np.asarray([theta, u[0], u[1]])
    
    @staticmethod
    def inv(X):
        R_inv = SO2.inv(X[0:2, 0:2])
        t = X[0:2, 2]
        t_inv = -R_inv @ t
        
        return np.asarray([[R_inv[0,0], R_inv[0,1], t_inv[0]],
                           [R_inv[1,0], R_inv[1,1], t_inv[1]],
                           [0.0, 0.0, 1.0]])

In [ ]:
# TODO

class so3(object):
    def __init__(self):
        pass
    
    @staticmethod
    def exp(theta):
        pass

class SO3(object):
    def __init__(self):
        pass
    
    @staticmethod
    def log(X):
        pass
    
    @staticmethod
    def inv(X):
        return np.transpose(X)
    
    lie_algebra = so3
    


In [ ]:
# TODO

class se3(object):
    def __init__(self):
        pass
    
    @staticmethod
    def exp(theta):
        pass

class SE3(object):
    lie_algebra = se3
    
    def __init__(self):
        pass
    
    @staticmethod
    def log(X):
        pass
    
    @staticmethod
    def inv(X):
        pass
    


## Lie Group Interpolation Function

This function works with all of the Lie groups that we will consider in the course of our robotics work.

In [ ]:
def interpolate(G, A, B, t):
    return (G.lie_algebra).exp(t * G.log(B @ G.inv(A))) @ A

## Interpolate 2D Rotation Using Lie Groups

In [ ]:
fix, ax = plt.subplots(figsize=(6,6))
ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.grid(True)

@widgets.interact(alpha=(0.0, 1.0, 0.01))
def update(alpha = 0.0):
    [l.remove() for l in ax.collections]
    z1 = so2.exp(0.0)
    z2 = so2.exp(np.pi/2.0)
    Z = interpolate(SO2, z1, z2, alpha)
    z = Z @ np.array([1.0, 0.0])
    ax.scatter(z[0], z[1], color="C0")

In [ ]:
def draw_axes_2d(SE2_elt, ax):
    t = SE2_elt[0:2, 2]
    R = SE2_elt[0:2, 0:2]
    
    # draw the origin
    ax.scatter(t[0], t[1], color="C0")
    
    # draw the x-axis
    x_rot = R @ np.asarray([1.0, 0.0])
    
    # draw the y-axis
    y_rot = R @ np.asarray([0.0, 1.0])
    
    c = [(1,0,0,1), (0,1,0,1)]
    lc = mc.LineCollection([
        [(t[0], t[1]), (t[0] + x_rot[0], t[1] + x_rot[1])],
        [(t[0], t[1]), (t[0] + y_rot[0], t[1] + y_rot[1])]
    ], colors=c)
    
    ax.add_collection(lc)

## Interpolate 2D Pose Using Lie Groups

The function below takes a pose and a covariance and draws the ellipse that contains 95% of the data generated by a bivariate normal with mean=pose and covariance=cov.

In [ ]:
def draw_gaussian(pose, cov, ax):
    #ax = plt.gca()

    #plt.scatter(mean[0], mean[1], c="Orange")
    
    theta = SE2.log(pose)[0]
    w, v = np.linalg.eig(cov)
    
    idx = w.argsort()[::-1]
    w = w[idx]
    v = v[idx]
    
    ax.add_patch(Ellipse((pose[0,2], pose[1,2]), 
            2.0 * sqrt(5.991 * w[1]), 
            2.0 * sqrt(5.991 * w[0]), 
            np.rad2deg(theta - np.arctan2(v[1][1], v[1][0])), 
            linewidth=1, 
            fill=False ))


In [ ]:
# Example covariances
sigma1 = np.asarray([[0.5, 0.0],
                    [0.0, 0.2]])

sigma2 = np.asarray([[0.1, 0.0],
                     [0.0, 0.1]])

In [ ]:
# Interpolate the covariance using the log-Euclidean geodesic.
# See: http://proceedings.mlr.press/v37/huanga15.pdf for the geodesic.
# See also: https://pyriemann.readthedocs.io/en/latest for some Riemannian geometry.
def exp_cov(A):
    w, v = eig(A)
    return v @ np.diag(np.exp(w)) @ v.T

def log_cov(A):
    w, v = eig(A)
    return v @ np.diag(np.log(w)) @ v.T

def interp_cov(sigma1, sigma2, t):
    return exp_cov((1-t) * log_cov(sigma1) + t * log_cov(sigma2))

## Interpolate Both Poses and Covariances

In [ ]:
fix, ax = plt.subplots(figsize=(6,6))
ax.set_ylim([-10,10])
ax.set_xlim([-10,10])
ax.grid(True)

@widgets.interact(alpha=(0.0, 1.0, 0.01))
def update(alpha = 0.0):
    [l.remove() for l in ax.collections]
    [p.remove() for p in ax.patches]
    
    theta1 = np.pi/4.0
    pose1 = np.asarray([[np.cos(theta1), -np.sin(theta1), 0.0],
                        [np.sin(theta1),  np.cos(theta1), 0.0],
                        [0.0, 0.0, 1.0]])
    theta2 = np.pi/2.0
    pose2 = np.asarray([[np.cos(theta2), -np.sin(theta2), 5.0],
                        [np.sin(theta2),  np.cos(theta2), 5.0],
                        [0.0, 0.0, 1.0]])
    
    Z = interpolate(SE2, pose1, pose2, alpha)
    Sigma = interp_cov(sigma1, sigma2, alpha)
    
    draw_axes_2d(Z, ax)
    
    draw_gaussian(Z, Sigma, ax)